In [1]:
import pandas as pd
from database import Database
from datetime import datetime
from Filter.Market import Market

In [7]:
db = Database()
q_data=db.get_finance_report_q()
s_data = db.get_stock('2023/05/16','2023/09/16')
# mk_data = db.get_market_index('2023/05/16','2023/09/16')
# company_data =db.get_company_basic()

In [9]:
s_data.dtypes

日期          datetime64[ns]
證券代碼                 int64
開盤價(元)              object
最高價(元)              object
最低價(元)              object
收盤價(元)              object
成交量(千股)             object
5日均價(元)             object
10日均價(元)            object
20日均價(元)            object
60日均價(元)            object
5日均量                object
10日均量               object
20日均量               object
60日均量               object
dtype: object

In [10]:
s_data['開盤價(元)'] = pd.to_numeric(s_data['開盤價(元)'])

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "pandas\_libs\lib.pyx", line 2369, in pandas._libs.lib.maybe_convert_numeric
ValueError: Unable to parse string "1,040"

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\User\AppData\Roaming\Python\Python311\site-packages\IPython\core\interactiveshell.py", line 3460, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "C:\Users\User\AppData\Local\Temp\ipykernel_11356\2487781320.py", line 1, in <module>
    s_data['開盤價(元)'] = pd.to_numeric(s_data['開盤價(元)'])
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\User\AppData\Local\Programs\Python\Python311\Lib\site-packages\pandas\core\tools\numeric.py", line 185, in to_numeric
    values, _ = lib.maybe_convert_numeric(
                ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "pandas\_libs\lib.pyx", line 2411, in pandas._libs.lib.maybe_convert_numeric
ValueError: Unable to parse string "1,040" a